1) 뉴스 기사 제목 데이터에 대한 이해


In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))


샘플의 수 : 11314


In [2]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [3]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


2) 텍스트 전처리

In [4]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

news_df['clean_doc'][1]


"yeah, expect people read faq, etc. actually accept hard atheism? need little leap faith, jimmy. your logic runs steam! jim, sorry can't pity you, jim. sorry that have these feelings denial about faith need well, just pretend that will happily ever after anyway. maybe start newsgroup, alt.atheist.hard, won't bummin' much? bye-bye, jim. don't forget your flintstone's chewables! bake timmons,"

In [5]:
import nltk
nltk.download('stopwords')

# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

print(tokenized_doc[:5])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    [well, sure, story, seem, biased., disagree, s...
1    [yeah,, expect, people, read, faq,, etc., actu...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal,,...
4    [well,, change, scoring, playoff, pool., unfor...
Name: clean_doc, dtype: object


3) 정수 인코딩과 단어 집합 만들기

In [6]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0


[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)]


In [7]:
import gensim
import scipy

print("gensim 버전:", gensim.__version__)
print("scipy 버전:", scipy.__version__)

gensim 버전: 4.3.2
scipy 버전: 1.10.1


In [8]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased., disagree, s...
1    [yeah,, expect, people, read, faq,, etc., actu...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal,,...
4    [well,, change, scoring, playoff, pool., unfor...
Name: clean_doc, dtype: object

In [9]:
print(dictionary[66])

bye-bye,


In [10]:
len(dictionary)

181856

In [11]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.010*"went" + 0.009*"israel" + 0.007*"said," + 0.006*"came"')
(1, '0.032*"space" + 0.009*"nasa" + 0.008*"launch" + 0.007*"satellite"')
(2, '0.014*"period" + 0.011*"power" + 0.008*"---------------" + 0.008*"scorer"')
(3, '0.007*"good" + 0.007*"like" + 0.005*"back" + 0.004*"bike"')
(4, '0.014*"team" + 0.014*"game" + 0.011*"----------------------------------------------------------------------" + 0.009*"games"')
(5, '0.222*"max>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'" + 0.005*"candida" + 0.002*"part" + 0.002*"--------"')
(6, '0.004*"rider" + 0.004*"macs" + 0.003*"copies)" + 0.003*"annual"')
(7, '0.008*"gordon" + 0.007*"banks" + 0.007*"----------------------------------------------------------------------------" + 0.007*"surrender"')
(8, '0.004*"price:" + 0.003*"0.333" + 0.003*"fleet" + 0.003*"retail"')
(9, '0.016*"55.0" + 0.002*"approval" + 0.002*"answer:" + 0.002*"null,"')
(10, '0.025*"drive" + 0.017*"disk" + 0.014*"scsi" + 0.013*"hard"')
(11, '0.01

In [12]:
print(ldamodel.print_topics())

[(0, '0.010*"went" + 0.009*"israel" + 0.007*"said," + 0.006*"came" + 0.006*"israeli" + 0.006*"said" + 0.006*"killed" + 0.005*"told" + 0.005*"started" + 0.005*"people"'), (1, '0.032*"space" + 0.009*"nasa" + 0.008*"launch" + 0.007*"satellite" + 0.006*"lunar" + 0.005*"shuttle" + 0.004*"flight" + 0.004*"water" + 0.004*"station" + 0.004*"rocket"'), (2, '0.014*"period" + 0.011*"power" + 0.008*"---------------" + 0.008*"scorer" + 0.008*"play:" + 0.005*"third" + 0.005*"second" + 0.005*"first" + 0.004*"pens" + 0.004*"quebec"'), (3, '0.007*"good" + 0.007*"like" + 0.005*"back" + 0.004*"bike" + 0.004*"power" + 0.004*"around" + 0.003*"i\'ve" + 0.003*"right" + 0.003*"little" + 0.003*"much"'), (4, '0.014*"team" + 0.014*"game" + 0.011*"----------------------------------------------------------------------" + 0.009*"games" + 0.008*"play" + 0.008*"hockey" + 0.008*"last" + 0.007*"players" + 0.007*"league" + 0.006*"year"'), (5, '0.222*"max>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\

In [13]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [14]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(11, 0.43244863), (14, 0.19756576), (17, 0.15136269), (18, 0.20644678)]
1 번째 문서의 topic 비율은 [(1, 0.023546906), (3, 0.2017912), (11, 0.1298445), (18, 0.62568885)]
2 번째 문서의 topic 비율은 [(0, 0.06571317), (3, 0.021684071), (11, 0.7767987), (12, 0.017593896), (13, 0.034406558), (14, 0.07191388)]
3 번째 문서의 topic 비율은 [(3, 0.06725724), (11, 0.68182254), (13, 0.010826313), (19, 0.22856699)]
4 번째 문서의 topic 비율은 [(4, 0.68220794), (10, 0.03776305), (11, 0.24848604)]


In [21]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table._append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)


In [22]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,11,0.4325,"[(11, 0.43249172), (14, 0.1975659), (17, 0.151..."
1,1,18,0.6256,"[(1, 0.023546906), (3, 0.20177624), (11, 0.129..."
2,2,11,0.7768,"[(0, 0.06566476), (3, 0.021683991), (11, 0.776..."
3,3,11,0.6818,"[(3, 0.067283005), (11, 0.68179876), (13, 0.01..."
4,4,4,0.6822,"[(4, 0.6821747), (10, 0.037763026), (11, 0.248..."
5,5,11,0.4131,"[(1, 0.23964219), (2, 0.04946118), (11, 0.4131..."
6,6,11,0.7794,"[(7, 0.013137767), (8, 0.013872294), (10, 0.12..."
7,7,11,0.5960,"[(3, 0.022998117), (4, 0.05374619), (11, 0.596..."
8,8,11,0.5136,"[(0, 0.28001803), (4, 0.031985622), (11, 0.513..."
9,9,11,0.5530,"[(3, 0.29164526), (4, 0.017296419), (7, 0.0147..."
